In [5]:
# Import MySQL Connector/Python 
import mysql.connector as connector

connection=connector.connect(user="root",password="Dynamopika88")

cursor = connection.cursor()

cursor.execute("CREATE DATABASE little_lemon_db") 

In [6]:


cursor.execute("USE little_lemon_db")

#MenuItems table
create_customers_table = """CREATE TABLE Customers (
CustomerID INT AUTO_INCREMENT,
FullName VARCHAR(200),
ContactNumber VARCHAR(100),
Email VARCHAR(200),
PRIMARY KEY (CustomerID)
);"""


create_booking_table = """CREATE TABLE Bookings (
BookingID INT AUTO_INCREMENT,
BookingDate DATE,
TableNumber INT,
CustomerID INT,
PRIMARY KEY (BookingID)
);"""


# Create Customer table
cursor.execute(create_customers_table)

# Create Booking table
cursor.execute(create_booking_table)












In [8]:
#*******************************************************#
# Insert query to populate "Bookings" table:
#*******************************************************#
insert_bookings="""
INSERT INTO Bookings (BookingID, BookingDate, TableNumber, CustomerID)
VALUES
(1,'2022-10-10',5,1),
(2,'2022-11-12',3,3),
(3,'2022-10-11',2,2),
(4,'2022-10-13',2,1);
"""


# Populate Bookings table
cursor.execute(insert_bookings)
connection.commit()


In [19]:
stored_procedure_query = """
CREATE PROCEDURE CheckBooking(IN datebooking DATE, IN tablenum INT)
BEGIN 
    DECLARE booking_exists INT;
    SELECT COUNT(*) INTO booking_exists
    FROM Bookings
    WHERE BookingDate = datebooking AND TableNumber = tablenum;
    IF booking_exists > 0 THEN
        SELECT CONCAT('Table ', tablenum, ' is already booked') AS 'Booking status';
    ELSE
        SELECT CONCAT('Table ', tablenum, ' is available') AS 'Booking status';
    END IF;
END
"""

cursor.execute(stored_procedure_query)


In [21]:
# Call the stored procedure with its name
cursor.callproc("CheckBooking",('2022-11-12',3))

# Retrieve recrods in "dataset"
results = next( cursor.stored_results() )
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data 
for data in dataset:
    print(data)

['Booking status']
('Table 3 is already booked',)


In [57]:
stored_procedure_query = """



CREATE PROCEDURE AddValidBooking(IN booking_date DATE, IN table_number INT)
BEGIN
    -- Declare variables
    DECLARE table_exists INT;

    -- Check if the table is already booked on the given date
    SELECT COUNT(*) INTO table_exists
    FROM Bookings
    WHERE BookingDate = booking_date AND TableNumber = table_number;

    -- Start a transaction
    START TRANSACTION;

    -- Check the result and decide to commit or rollback
    IF table_exists > 0 THEN
        -- If the table is already booked, display a message and rollback the transaction
        SELECT CONCAT('Table ', table_number, ' is already booked - booking cancelled') AS 'Booking status';
        -- Rollback the transaction
        ROLLBACK;
    ELSE
        -- If the table is available, perform the booking insertion
        INSERT INTO Bookings(BookingDate, TableNumber)
        VALUES (booking_date, table_number);
        
        -- Display a message and commit the transaction
        SELECT CONCAT('Table ', table_number, ' is available for booking.') AS 'Booking status';
        COMMIT;
    END IF;
END
"""

cursor.execute(stored_procedure_query)


In [58]:
# Call the stored procedure with its name
cursor.callproc("AddValidBooking",('2022-11-12',3))

# Retrieve recrods in "dataset"
results = next( cursor.stored_results() )
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data 
for data in dataset:
    print(data)

['Booking status']
('Table 3 is already booked - booking cancelled',)


In [80]:
sql = "DROP PROCEDURE IF EXISTS CandelBooking;"

cursor.execute(sql)

In [65]:
stored_procedure_query = """
CREATE PROCEDURE AddBooking(IN booking_id INT, IN customer_id INT, IN booking_date DATE, IN table_num INT)
BEGIN
    DECLARE booking_message VARCHAR(255);

    -- Perform the booking insertion
    INSERT INTO Bookings(BookingID, BookingDate, TableNumber, CustomerID)
    VALUES (booking_id, booking_date, table_num, customer_id);

    -- Check if the booking was successful
    IF ROW_COUNT() > 0 THEN
        SET booking_message = 'New booking added';
    ELSE
        SET booking_message = CONCAT('Failed to add booking ', booking_id, '.');
    END IF;

    -- Return the booking message
    SELECT booking_message AS 'Booking status';
END
"""
cursor.execute(stored_procedure_query)

In [69]:
# Call the stored procedure with its name
cursor.callproc("AddBooking",(40,2,'2022-11-12',3))

# Retrieve recrods in "dataset"
results = next( cursor.stored_results() )
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data 
for data in dataset:
    print(data)

['Booking status']
('New booking added',)


In [73]:
stored_procedure_query = """
CREATE PROCEDURE UpdateBooking(IN booking_id INT,IN booking_date DATE)
BEGIN
    DECLARE booking_exists INT;
    SELECT COUNT(*) INTO booking_exists FROM Bookings WHERE BookingID = booking_id;
    IF booking_exists > 0 THEN
        -- Update the booking date
        UPDATE Bookings
        SET BookingDate = booking_date
        WHERE BookingID = booking_id;

        SELECT CONCAT('Booking ', booking_id, ' updated .') AS 'Update status';
    ELSE
        SELECT CONCAT('Booking ', booking_id, ' does not exist. No update performed.') AS 'Update status';
    END IF;
END
"""
cursor.execute(stored_procedure_query)

In [75]:
# Call the stored procedure with its name
cursor.callproc("UpdateBooking",(4100,'2022-11-30'))

# Retrieve recrods in "dataset"
results = next( cursor.stored_results() )
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data 
for data in dataset:
    print(data)

['Update status']
('Booking 4100 does not exist. No update performed.',)


In [81]:
stored_procedure_query = """
CREATE PROCEDURE CandelBooking(IN booking_id INT)
BEGIN
    DECLARE booking_exists INT;
    SELECT COUNT(*) INTO booking_exists FROM Bookings WHERE BookingID = booking_id;
    IF booking_exists > 0 THEN
        -- Update the booking date
        DELETE FROM Bookings
        WHERE BookingID = booking_id;

        SELECT CONCAT('Booking ', booking_id, ' cancelled .') AS 'Cancellation status';
    ELSE
        SELECT CONCAT('Booking ', booking_id, ' does not exist. No cancellation performed.') AS 'Cancellation status';
    END IF;
END
"""
cursor.execute(stored_procedure_query)

In [83]:
# Call the stored procedure with its name
cursor.callproc("CandelBooking",(1,))

# Retrieve recrods in "dataset"
results = next( cursor.stored_results() )
dataset = results.fetchall()

# Retrieve column names using list comprehension in a 'for' loop 
for column_id in cursor.stored_results():
    columns = [ column[0] for column in column_id.description ]

# Print column names
print(columns)

# Print data 
for data in dataset:
    print(data)

['Cancellation status']
('Booking 1 cancelled .',)
